# Colab-deepfillv2

A colab of [JiahuiYu/generative_inpainting](https://github.com/JiahuiYu/generative_inpainting).

My fork is locared in [styler00dollar/Colab-generative-inpainting](https://github.com/styler00dollar/Colab-generative-inpainting).

Currently only testing works properly. Training is WIP.

In [ ]:
#@title Check GPU
!nvidia-smi

In [ ]:
#@title Install
%cd /content
!pip install tensorflow==1.7.0
!pip install git+https://github.com/JiahuiYu/neuralgym

# Testing

In [ ]:
#@title Download git and models
%cd /content/
!git clone -b v2.0.0 https://github.com/JiahuiYu/generative_inpainting
%cd /content/generative_inpainting
!mkdir model_logs
%cd model_logs
!pip install gdown
# celebHQ
!mkdir /content/generative_inpainting/model_logs/release_celeba_hq_256_deepfill_v2
%cd /content/generative_inpainting/model_logs/release_celeba_hq_256_deepfill_v2
!gdown --id 1C7kPxqrpNpQF7B2UAKd_GvhUMk0prRdV
!gdown --id 1ExY4hlx0DjVElqJlki57la3Qxu40uhgd
!gdown --id 1z9dbEAzr5lmlCewixevFMTVBmNuSNAgK
!gdown --id 1dyPD2hx0JTmMuHYa32j-pu--MXqgLgMy
# places
!mkdir /content/generative_inpainting/model_logs/release_places2_256_deepfill_v2
%cd /content/generative_inpainting/model_logs/release_places2_256_deepfill_v2
!gdown --id 1C7kPxqrpNpQF7B2UAKd_GvhUMk0prRdV
!gdown --id 1ExY4hlx0DjVElqJlki57la3Qxu40uhgd
!gdown --id 1z9dbEAzr5lmlCewixevFMTVBmNuSNAgK
!gdown --id 1dyPD2hx0JTmMuHYa32j-pu--MXqgLgMy

Place pictures in ```/content/image.png``` and ```/content/mask.png```. Result will be in ```/content/result.png```.

Info: The mask is a black white image and the masked area is white.

In [ ]:
#@title Run test
%cd /content/generative_inpainting
!python test.py --image /content/input.png --mask /content/mask.png --output /content/output.png --checkpoint_dir model_logs/release_places2_256_deepfill_v2

# Training (currently experimental, it may or may not work)

In [ ]:
#@title Creation of .flist generation (needs to be tested)
%%writefile /content/generative_inpainting/prepare.py
#!/usr/bin/python

import argparse
import os
from random import shuffle

parser = argparse.ArgumentParser()
parser.add_argument('--folder_path', default='./training_data', type=str,
                    help='The folder path')
parser.add_argument('--train_filename', default='./data_flist/train_shuffled.flist', type=str,
                    help='The output filename.')
parser.add_argument('--validation_filename', default='./data_flist/validation_shuffled.flist', type=str,
                    help='The output filename.')
parser.add_argument('--is_shuffled', default='1', type=int,
                    help='Needed to shuffle')

if __name__ == "__main__":

    args = parser.parse_args()

    # get the list of directories
    dirs = os.listdir(args.folder_path)
    dirs_name_list = []

    # make 2 lists to save file paths
    training_file_names = []
    validation_file_names = []

    # print all directory names
    for dir_item in dirs:
        # modify to full path -> directory
        dir_item = args.folder_path + "/" + dir_item
        # print(dir_item)

        training_folder = os.listdir('/content/generative_inpainting/training_data')
        for training_item in training_folder:
            #training_item = dir_item + "/training" + "/" + training_item
            training_file_names.append(training_item)

        validation_folder = os.listdir('/content/generative_inpainting/validation_data')
        for validation_item in validation_folder:
            #validation_item = dir_item + "/validation" + "/" + validation_item
            validation_file_names.append(validation_item)
    # print all file paths
    for i in training_file_names:
        print(i)
    for i in validation_file_names:
        print(i)

    # This would print all the files and directories

    # shuffle file names if set
    if args.is_shuffled == 1:
        shuffle(training_file_names)
        shuffle(validation_file_names)

    # make output file if not existed
    if not os.path.exists(args.train_filename):
        os.mknod(args.train_filename)

    if not os.path.exists(args.validation_filename):
        os.mknod(args.validation_filename)

    # write to file
    fo = open(args.train_filename, "w")
    fo.write("\n".join(training_file_names))
    fo.close()

    fo = open(args.validation_filename, "w")
    fo.write("\n".join(validation_file_names))
    fo.close()

    # print process
    print("Written file is: ", args.train_filename, ", is_shuffle: ", args.is_shuffled)

In [ ]:
#@title Running .flist generation
%cd /content/generative_inpainting
!python /content/generative_inpainting/prepare.py --folder_path /content/generative_inpainting/validation_data --train_filename /content/generative_inpainting/training_data/train.flist --validation_filename /content/generative_inpainting/validation_data/val.flist

In [ ]:
#@title Creating config
%%writefile /content/generative_inpainting/inpaint.yml
# =========================== Basic Settings ===========================
# machine info
num_gpus_per_job: 1  # number of gpus each job need
num_cpus_per_job: 2  # number of gpus each job need
num_hosts_per_job: 1
memory_per_job: 32  # number of gpus each job need
gpu_type: 'nvidia-tesla-p100'

# parameters
name: places2_gated_conv_v100  # any name
model_restore: ''  # logs/places2_gated_conv
dataset: 'celebahq'  # 'tmnist', 'dtd', 'places2', 'celeba', 'imagenet', 'cityscapes'
random_crop: True  # Set to false when dataset is 'celebahq', meaning only resize the images to img_shapes, instead of crop img_shapes from a larger raw image. This is useful when you train on images with different resolutions like places2. In these cases, please set random_crop to true.
val: True  # true if you want to view validation results in tensorboard
log_dir: logs/full_model_celeba_hq_256

gan: 'sngan'
gan_loss_alpha: 1
gan_with_mask: True
discounted_mask: True
random_seed: False
padding: 'SAME'

# training
train_spe: 10
max_iters: 100000000
viz_max_out: 10
val_psteps: 2000

# data
data_flist:
  # https://github.com/jiahuiyu/progressive_growing_of_gans_tf
  celebahq: [
    '/content/generative_inpainting/training_data/train.flist',
    '/content/generative_inpainting/validation_data/val.flist'
  ]

static_view_size: 30
img_shapes: [256, 256, 3]
height: 128
width: 128
max_delta_height: 32
max_delta_width: 32
batch_size: 4
vertical_margin: 0
horizontal_margin: 0

# loss
ae_loss: True
l1_loss: True
l1_loss_alpha: 1.

# to tune
guided: False
edge_threshold: 0.6


In [ ]:
#@title Getting tensorboard link (needs to be tested)
LOG_DIR = '/content/generative_inpainting/logs/full_model_celeba_hq_256'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Seems like progress printing got removed according to [this](https://github.com/JiahuiYu/generative_inpainting/issues/358). Don't panic. The official recommendation is to check GPU usage, but that is a bit hard in colab. If checkpoints get generated, then you can be sure it somehow works.

In [ ]:
#@title Starting training
!python /content/generative_inpainting/train.py